In [6]:
import subprocess
import os

not_small_video_files = [f.lower().split(".mp4")[0] for f in os.listdir("videos/") if f[0] != "." and "small" not in f]
small_video_files = [f.lower().split("-small.mp4")[0] for f in os.listdir("videos/") if f[0] != "." and "small" in f]
videos_to_process = [n for n in not_small_video_files if n not in small_video_files]

print not_small_video_files
print small_video_files
print videos_to_process

for video in videos_to_process: 
    print "processing video", video
    fn_in = "videos/" + video + ".mp4"
    fn_out = "videos/" + video + "-small.mp4"
    
    # ffmpeg -i nocuts.MP4 -vf scale=500:-1 nocuts-small.MP4
    # subprocess.call(["ffmpeg", "-i", fn_in, "-vf", "scale=500:-1", fn_out])

['arctic', 'congo_2048', 'equation-2', 'equation', 'gittes-2', 'gittes-reg-2', 'gittes-reg', 'invasion', 'nocuts', 'probation-reg', 'probation-shoulder', 'steve-walking', 'toy-story-shoulder-2', 'toy-story-shoulder', 'toy-story']
['arctic', 'congo_2048', 'equation-2', 'equation', 'gittes-2', 'gittes-reg-2', 'gittes-reg', 'invasion', 'nocuts', 'nocuts-smaller.mp4', 'probation-reg', 'probation-shoulder', 'steve-walking', 'toy-story-shoulder-2', 'toy-story-shoulder', 'toy-story']
[]


In [7]:
import numpy as np
import cv2
from collections import defaultdict
import json

# attempting optical flow on one 360 video
# sparse features 
# from - http://docs.opencv.org/3.2.0/d7/d8b/tutorial_py_lucas_kanade.html

def save_important_points_fn(fn, recorded_important_points):
    
    out_fn = "analysis/optical-flow/" + fn.lower().split(".mp4")[0].split("/")[-1] + ".json"
    print "Saving to", out_fn
    print recorded_important_points[0:20]

    new_points = []
    for p in recorded_important_points:
        p["movement"] = float(p["movement"])
        p["a"] = [float(p["a"][0]), float(p["a"][1])]
        p["b-a"] = [float(p["b-a"][0]), float(p["b-a"][1])]
        p["b"] = [float(p["b"][0]), float(p["b"][1])]
        new_points.append(p)

    with open("analysis/optical-flow/" + fn.lower().split(".mp4")[0].split("/")[-1] + ".json", 'w') as f:
        json.dump(new_points, f)

def find_important_points_for_fn(fn):
    cap = cv2.VideoCapture(fn)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.3,
                           minDistance = 7,
                           blockSize = 7 )
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create some random colors
    color = np.random.randint(0,255,(100,3))

    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    def dist(x,y):   
        return np.sqrt(np.sum((x-y)**2))

    frame_count = 0
    feature_sums = defaultdict(float)
    feature_counts = defaultdict(float)
    feature_initial_values = defaultdict(list)
    recorded_important_points = []

    while(1):
        frame_count += 1

        ret,frame = cap.read()
        
        if frame != None and frame.any(): 
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            good_new = p1[st==1]
            good_old = p0[st==1]

            for i,(new,old) in enumerate(zip(good_new,good_old)):

                a,b = new.ravel()
                c,d = old.ravel()
                mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

                # added to record important points
                diff = dist(new, old)

                key = str(i)
                feature_sums[key] += diff
                feature_counts[key] += 1.0

                if frame_count % 29 == 0 and frame_count != 0:
                    movement = feature_sums[key]/feature_counts[key]
                    b_minus_a = new-feature_initial_values[key]

                    # if abs(b_minus_a[0]) >= 1.0 or abs(b_minus_a[1]) >= 1.0:
                    point = {
                            "b": new.tolist(), 
                            "a": feature_initial_values[key].tolist(),
                            "b-a": b_minus_a.tolist(),
                            "feature": key,
                            "frame": frame_count,
                            "movement": movement,
                            "time": frame_count/fps
                        }
                    recorded_important_points.append(point)

                    feature_initial_values[key] = new

                if frame_count == 1: 
                    feature_initial_values[key] = new

            img = cv2.add(frame,mask)
            cv2.imshow('frame',img)

            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break

            # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1,1,2)
        else:
            return recorded_important_points
    return recorded_important_points
        
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cap.release()

In [8]:
import time

result_root = "analysis/optical-flow/"
small_video_files = [f for f in os.listdir("videos/") if f[0] != "." and "small" in f][2:]
# result_files = [m.split('.json')[0] + ".mp4" for m in os.listdir(result_root) if m[0] != "."]
result_files = [] # need to reprocess all of the files
videos_to_process = [s for s in small_video_files if s.lower() not in result_files and "invasion" not in s]
print videos_to_process

for video_fn in videos_to_process: 
    fn = "videos/" + video_fn
    print "Processing:", fn
    
    start = time.time()
    recorded_important_points = find_important_points_for_fn(fn)
    save_important_points_fn(fn, recorded_important_points)
    print "Minutes to process:", (time.time() - start)/60.0

['equation-2-small.mp4', 'equation-small.mp4', 'gittes-2-small.mp4', 'gittes-reg-2-small.mp4', 'gittes-reg-small.mp4', 'nocuts-small.MP4', 'nocuts-smaller.mp4', 'probation-reg-small.mp4', 'probation-shoulder-small.mp4', 'steve-walking-small.mp4', 'toy-story-shoulder-2-small.mp4', 'toy-story-shoulder-small.mp4', 'toy-story-small.mp4']
Processing: videos/equation-2-small.mp4
Saving to analysis/optical-flow/equation-2-small.json
[{'a': [215.98696899414062, 127.0182113647461], 'b': [216.0643768310547, 126.94255828857422], 'frame': 29, 'feature': '0', 'b-a': [0.0774078369140625, -0.075653076171875], 'time': 0.9860000000000001, 'movement': 0.019925923212652158}, {'a': [215.96563720703125, 150.0102081298828], 'b': [216.0249481201172, 149.9840087890625], 'frame': 29, 'feature': '1', 'b-a': [0.0593109130859375, -0.0261993408203125], 'time': 0.9860000000000001, 'movement': 0.027426232262675106}, {'a': [453.9976806640625, 138.0035400390625], 'b': [453.99493408203125, 138.00169372558594], 'frame':

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Processing: videos/equation-2-small.mp4
Saving to analysis/optical-flow/equation-2-small.json
[{'a': [216.0580291748047, 126.96463012695312], 'b': [218.3977813720703, 129.02935791015625], 'frame': 87, 'feature': '0', 'b-a': [2.339752197265625, 2.064727783203125], 'time': 2.958, 'movement': 0.08474402544245889}, {'a': [216.2547149658203, 149.899658203125], 'b': [234.08021545410156, 147.72576904296875], 'frame': 87, 'feature': '1', 'b-a': [17.82550048828125, -2.17388916015625], 'time': 2.958, 'movement': 0.23165659439795511}, {'a': [232.00070190429688, 133.00161743164062], 'b': [237.9292755126953, 128.9722442626953], 'frame': 87, 'feature': '16', 'b-a': [5.9285736083984375, -4.0293731689453125], 'time': 2.958, 'movement': 0.099226436310115906}, {'a': [205.16656494140625, 159.39088439941406], 'b': [207.86172485351562, 158.98165893554688], 'frame': 87, 'feature': '24', 'b-a': [2.695159912109375, -0.4092254638671875], 'time': 2.958, 'movement': 0.062072473449696755}, {'a': [242.090026855468

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


KeyboardInterrupt: 

In [ ]:
# hog pedestrian detection from - (http://arijitgeek.me/index.php/201
# 6/06/26/opencv-python-people-detect-in-video-feed/)
# lots of false positives!!

import numpy as np
import cv2

fn = "videos/equation-2.mp4"

def draw_detections(img, rects, thickness = 1):
    for x, y, w, h in rects:
        # the HOG detector returns slightly larger rectangles than the real objects.
        # so we slightly shrink the rectangles to get a nicer output.
        pad_w, pad_h = int(0.15*w), int(0.05*h)
        cv2.rectangle(img, (x+pad_w, y+pad_h), (x+w-pad_w, y+h-pad_h), (0, 255, 0), thickness)
    
hog = cv2.HOGDescriptor()
hog.setSVMDetector( cv2.HOGDescriptor_getDefaultPeopleDetector() )
cap=cv2.VideoCapture(fn)
while True:
    _,frame=cap.read()
    found,w=hog.detectMultiScale(frame, winStride=(8,8), padding=(32,32), scale=1.05)
    draw_detections(frame,found)
    cv2.imshow('feed',frame)
    ch = 0xFF & cv2.waitKey(30)
    if ch == 27:
      break
cv2.destroyAllWindows()

# dense features
# from - http://docs.opencv.org/3.2.0/d7/d8b/tutorial_py_lucas_kanade.html

fn = "videos/equation-2-small.mp4"

import cv2
import numpy as np
cap = cv2.VideoCapture(fn)
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv2.destroyAllWindows()

In [23]:
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)

-1